# 2.3 Formatos de datos

En este tutorial, manipularemos la estructura de datos desde y hacia varios formatos de datos.


Los formatos que soportan datos no estructurados (no relacionales) son:
- JSON: JavaScript Object Notation, un formato de archivo estándar abierto que utiliza texto legible por humanos. Los datos pueden ser pares atributo-valor y matrices. Es independiente del lenguaje. La sintaxis es la siguiente
```
{
  "nombre": "John",
  "apellido": "Smith",
  "estaVivo": verdadero,
  "edad": 27,
  "dirección": {
    "calle": "21 2nd Street",
    "ciudad": "New York",
    "estado": "NY",
    "CodigoPostal": "10021-3100"
  }
```
La codificación de caracteres es UTF-8. Los tipos de datos en archivos JSON pueden ser números, cadena, booleano, matriz, obkect (colección de pares nombre-valor), null. Más información sobre JSON en el [curso EarthDataScience](!https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/apis-in-python/).


Los principales formatos que admiten datos ráster pixelizados son:
- 
**GeoTIFF**: norma de metadatos que permite incluir información de georreferenciación en un archivo TIFF (Tagged Image File Format).  **GeoTIFF** está mejorado para ser optimizado para la nube.
- GeoJSON**: GeoJSON es un formato para codificar una variedad de estructuras de datos geográficos en el formato JSON.


Los formatos que admiten datos tabulares son
- CSV
- Parquet


Los formatos de datos para grandes datos heterogéneos (diferentes tipos de datos):
- NetCDF4
- HDF5
- Zarr

In [ ]:
import requests, zipfile , os, io
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import pandas as pd
# import pycrs
import rasterio
import h5py
import rasterio
import netCDF4 as nc
import wget


from folium.plugins import MarkerCluster
from rasterio.mask import mask
from rasterio.plot import show

## 1. Datos raster

### 1.1 rasterio para leer GeoTIFF

Los datos raster son datos pixelados (o cuadriculados) en los que cada píxel está asociado a una ubicación geográfica específica. El valor de un píxel puede ser continuo (por ejemplo, la elevación) o categórico (por ejemplo, el uso del suelo).

El paquete python ``rasterio``, con documentación [aquí](!https://rasterio.readthedocs.io/en/latest/), y que puede leer formatos como ``GeoTIFF`` y ``GeoJSON``.

Ver materiales introductorios adicionales de [EarthDataScience](!https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/), y tutoriales de la [GeoHackweek](!https://geohackweek.github.io/raster/).



Descargaremos archivos de topografía que se encuentran en esta [página](!https://www.naturalearthdata.com/downloads/50m-raster-data/50m-cross-blend-hypso/), pero almacenados en una carpeta de Dropbox.

El nombre del archivo es `HYP_50M_SR` y es un archivo comprimido.



In [ ]:
# Descarga los datos utilizando wget.
fname = 'HYP_50M_SR'
wget.download("https://www.dropbox.com/s/r75ecms0bvyqaca/"+str(fname)+"?dl=1") # anote el último carácter como cadena para solicitar el propio archivo

El archivo de datos se guardará en el directorio de inicio, queremos moverlo a una carpeta ``data``:

In [ ]:
os.replace(fname+".zip", './data/'+fname)

Descomprimir el archivo

In [ ]:

os.makedirs("./data/"+fname+"/",exist_ok=True)
# wget.download(url,out="HYP_50M_SR") # this does not work on the hub
z = zipfile.ZipFile('./data/'+fname+".zip")
z.extractall("./data/"+fname+"/")

Ahora vamos a obtener el Mapa Digital de Elevaciones. Abrimos el archivo descomprimido utilizando el paquete ``rasterio``.

In [ ]:
elevation = rasterio.open("./data/"+fname+"/"+fname+".tif")

In [ ]:
print(elevation.variables.keys())

Veamos las dimensiones de los datos:

In [ ]:
elevation.height

In [ ]:
elevation.width

In [ ]:
elevation.indexes

¿Puedes adivinar cómo llamar a los tipos de datos de la entrada del archivo?

In [1]:
# escribe abajo

y en los límites del conjunto de datos:

In [ ]:
elevation.bounds

In [ ]:
print(elevation.transform * (0, 0)) # North West corner
print(elevation.transform * (elevation.width, elevation.height)) # South East corner

He aquí la proyección utilizada para los datos:

In [ ]:
elevation.crs

Cómo interpretar los datos: Hay tres capas para los tres colores rojo, verde y azul:

In [ ]:
print(elevation.colorinterp[0])
print(elevation.colorinterp[1])
print(elevation.colorinterp[2])

In [ ]:
print(np.min(elevation.read(1)), np.max(elevation.read(1)))
print(np.min(elevation.read(2)), np.max(elevation.read(2)))
print(np.min(elevation.read(3)), np.max(elevation.read(3)))

Grafiquemos los datos

In [ ]:
image = elevation.read()

In [ ]:
show(image)

### 1.2 Geopandas para leer GeoJSON

Los GeoTIFF no son el único tipo de ficheros que podemos leer con geopandas. Veamos un ejemplo de lectura de datos de un fichero geojson (que es un caso especial de fichero json con coordenadas geográficas).

In [ ]:
url = 'https://www.nps.gov/lib/npmap.js/4.0.0/examples/data/national-parks.geojson'

In [ ]:
parks = gpd.read_file(url)

In [ ]:
parks.head()

Vamos a graficar los datos.

folium es un buen paquete de Python para la visualización. El [Geohackweek tutorial on Folium](!https://github.com/geohackweek/tutorial_contents/blob/master/visualization/notebooks/foliumTutorial.ipynb) también es informativo.


In [ ]:
m = folium.Map(location=[40, -100], zoom_start=4)
folium.GeoJson(parks).add_to(m)
marker_cluster = MarkerCluster().add_to(m)
m

Vamos a centrarnos en los parques del Estado de Washington:

In [ ]:
parks_WA = parks.iloc[[94, 127, 187, 228, 286, 294, 295, 297, 299, 300, 302]].reset_index()

Creamos una lista de ubicaciones para añadir ventanas emergentes al mapa.

In [ ]:
locations = []
for index in range(0, len(parks_WA)):
    location = [parks_WA['geometry'][index].y, parks_WA['geometry'][index].x]
    locations.append(location)

In [ ]:
m = folium.Map(location=[47, -121], zoom_start=7)
marker_cluster = MarkerCluster().add_to(m)
for point in range(0, len(locations)):
    folium.Marker(location = locations[point], popup=parks_WA['Name'].iloc[point]).add_to(marker_cluster)
m

## 2. Formatos jerárquicos: NETCDF4 Y HDF5

Los formatos de datos jerárquicos están diseñados para almacenar grandes cantidades de datos en un único archivo. Imitan un sistema de archivos (por ejemplo, una estructura de datos en forma de árbol con directorios anidados) en un único archivo.  Existen dos formatos de datos jerárquicos dominantes (HDF5 y NETCDF4), y uno emergente para la nube (Zarr). Los formatos jerárquicos en general pueden almacenar muchos tipos de datos (numéricos frente a cadenas).

## HDF5
El Formato Jerárquico de Datos versión 5 (HDF5), es un formato de archivo de código abierto que soporta datos grandes, complejos y heterogéneos. HDF5 utiliza una estructura similar a un "directorio de archivos" que permite organizar los datos dentro del archivo de muchas formas estructuradas diferentes, como podría hacer con los archivos de su ordenador. El formato HDF5 también permite incrustar metadatos, por lo que es _autodescriptivo_.
Los archivos HDF5 son autodescriptivos, lo que significa que todos los elementos (el propio archivo, los grupos y los conjuntos de datos) pueden tener metadatos asociados que describen la información contenida en el elemento.


Ejemplo de estructura HDF:
- Conjuntos de datos, que son matrices multidimensionales tipadas
- Grupos, que son estructuras contenedoras que pueden contener conjuntos de datos y otros grupos


<img src="hdf5_structure4.jpeg" alt="Ilustración de un conjunto de datos H5 " />
Figura: Ejemplo de datos HDF5. Encontrado en [neonscience](https://www.neonscience.org/resources/learning-hub/tutorials/about-hdf5)

## Netcdf

El formulario común de datos de red, o **netCDF**, se creó a principios de los años 90 y se propuso resolver algunos de los problemas que planteaba trabajar con matrices N-dimensionales. Netcdf es una colección de formatos de datos binarios autodescriptivos e independientes de la máquina y de herramientas de software que facilitan la creación, el acceso y el intercambio de datos científicos almacenados en matrices N-dimensionales, junto con metadatos que describen el contenido de cada matriz. Netcdf fue creado por la comunidad científica del clima en un momento en que los modelos climáticos regionales empezaban a producir archivos de salida cada vez más grandes. La versión 4 de NetCDF es ahora un subconjunto de HDF5.

### Manejo de matrices de gran tamaño
Los formatos NetCDF y H5 no limitan el tamaño de los archivos. Sin embargo, cualquier herramienta de análisis que lea datos de una matriz NetCDF en memoria para alguna operación computacional estará limitada por la memoria disponible de esa máquina en particular. 

### Pero lento en I/O
Al leer un archivo jerárquico, se escanea todo el árbol de la estructura de datos desde el nodo raíz hacia abajo. Dado que hay que hacerlo cada vez que un usuario realiza una consulta, la lectura de H5 y Netcdf es **lenta**. Hay métodos más rápidos


Vamos a descargar un mapa geológico almacenado en formato netCDF en Dropbox. Los datos originales se pueden encontrar en la [base de datos USGS](!https://www.sciencebase.gov/catalog/item/5cfeb4cce4b0156ea5645056). (https://doi.org/10.3133/ofr20191081)

In [ ]:
# Descargar el marco geológico
file1 = wget.download("https://www.dropbox.com/s/wdb25puxh3u07dj/NCM_GeologicFrameworkGrids.nc?dl=1") #"./data/NCM_GeologicFrameworkGrids.nc"
# Descargar las rejillas de coordenadas
file2 = wget.download("https://www.dropbox.com/s/wdb25puxh3u07dj/NCM_SpatialGrid.nc?dl=1") #"./data/NCM_GeologicFrameworkGrids.nc"

In [ ]:
# mover los datos
os.replace(file1,'./data/'+file1)
os.replace(file2,'./data/'+file2)

In [ ]:
# leer los datos
geology = nc.Dataset('./data/'+file1)
grid = nc.Dataset('./data/'+file2)

In [ ]:
geology

In [ ]:
geology['Surface Elevation']

In [ ]:
np.shape(geology['Surface Elevation'])

In [ ]:
geology['Surface Elevation'][3246, 1234]

In [ ]:
x = grid['x'][0:4901, 0:3201]
y = grid['y'][0:4901, 0:3201]
elevation = geology['Surface Elevation'][0:4901, 0:3201]

In [ ]:
plt.contourf(x, y, elevation)

## 3. Zarr


Zarr es un formato de datos optimizado para la nube que maneja conjuntos de datos heterogéneos.

En el siguiente ejercicio, utilizaremos los conjuntos de datos abiertos air_temperature de Xarray y los guardaremos en un archivo Zarr.

Vamos a trabajar en grupos para :
- Descargar los datos xarray
- Guardarlos en un fichero, informar sobre el tiempo y el tamaño del conjunto de datos.
- Leerlo de nuevo y comprobar de nuevo el tiempo de escritura y los tiempos de lectura